# Part 1 - Fundamentals
## Chapter 2 - Passing code with behavior parameterization
You’ll learn about behavior parameterization, a software devel- opment pattern that Java 8 relies heavily on and is the motivation for lambda expressions.

In [1]:
System.out.println("Lets start with second chapter of this book, without wasting any time!")

Lets start with second chapter of this book, without wasting any time!


### This chapter covers
* Coping with changing requirements
* Behavior parameterization
* Anonymous classes
* Preview of lambda expressions
* Real-world examples: Comparator, Runnable, and GUI

In [2]:
public static class Apple {

	private Integer weight = 0;
	private String color = "";

	public Apple(int weight, String color) {
		this.weight = weight;
		this.color = color;
	}

	public Integer getWeight() {
		return weight;
	}

	public void setWeight(int weight) {
		this.weight = weight;
	}

	public String getColor() {
		return color;
	}

	public void setColor(String color) {
		this.color = color;
	}

	@Override
	public String toString() {
		return String.format("Apple{color='%s', weight=%d}", color, weight);
	}
}

__A well-known problem in software engineering is that no matter what you do, user requirements will change.__

For example, imagine an application to help a farmer understand his inventory. The farmer might want a functionality to find all green apples in his inventory. But the next day he might tell you, “Actually, I also want to find all apples heavier than 150 g.” Two days later, the farmer comes back and adds, “It would be really nice if I could find all apples that are green and heavier than 150 g.” How can you cope with these changing requirements? Ideally, you’d like to minimize your engineering effort.

Model your selection criteria: you’re working with apples and returning a boolean based on some attributes of Apple. For example, is it green? Is it heavier than 150 g? We call this a predicate (a function that returns a boolean). Let’s therefore define an interface to model the selection criteria:

In [3]:
public interface ApplePredicate{
    boolean test (Apple apple);
}

In [4]:
public class AppleHeavyWeightPredicate implements ApplePredicate {
    public boolean test(Apple apple) {
        return apple.getWeight() > 150; 
    }
}

public class AppleGreenColorPredicate implements ApplePredicate {
    public boolean test(Apple apple) {
        return "green".equals(apple.getColor());
    }
}

__ApplePredicate encapsulates a strategy for selecting an apple__

What you just did is related to the strategy design pattern (see http://en.wikipedia.org/wiki/Strategy_pattern), which lets you define a family of algorithms, encapsulate each algorithm (called a strategy), and select an algorithm at run time. 

This is what behavior parameterization means: the ability to tell a method to take multiple behaviors (or strategies) as parameters and use them internally to accomplish different behaviors.

In [5]:
public static List<Apple> filterApples(List<Apple> inventory, ApplePredicate p) {
	var result = new ArrayList<Apple>();
	for (Apple apple : inventory) {
		if (p.test(apple)) {
			result.add(apple);
		}
	}
	return result;
}

PASSING CODE/BEHAVIOR

In [6]:
public class AppleRedAndHeavyPredicate implements ApplePredicate {
        public boolean test(Apple apple){
			return "red".equals(apple.getColor()) && apple.getWeight() > 150;
		}
}

In [7]:
private List<Apple> getInventory() {
	var inventory = Arrays.asList(new Apple(80, "green"), 
                                  new Apple(255, "green"), 
                                  new Apple(220, "red"));
	return inventory;
}

var inventory = getInventory();
System.out.println(inventory);

[Apple{color='green', weight=80}, Apple{color='green', weight=255}, Apple{color='red', weight=220}]


In [8]:
List<Apple> redAndHeavyApples = filterApples(inventory, new AppleRedAndHeavyPredicate());

redAndHeavyApples

[Apple{color='red', weight=220}]

You’ve achieved something cool; the behavior of the filterApples method depends on the code you pass to it via the ApplePredicate object. You’ve parameterized the behavior of the filterApples method!

By using lambdas, you can directly pass the expression RED.equals(apple.getColor()) && apple.getWeight() > 150 to the filterApples method without having to define multiple ApplePredicate classes. This removes unnecessary verbosity.

Verbosity in general is bad; it discourages the use of a language feature because it takes a long time to write and maintain verbose code, and it’s not pleasant to read! Good code should be easy to comprehend at a glance.

Using a lambda expression

In [9]:
List<Apple> result = filterApples(inventory, (Apple apple) -> "red".equals(apple.getColor()));
result

[Apple{color='red', weight=220}]

You have to admit this code looks a lot cleaner than our previous attempts! It’s great because it’s starting to look a lot closer to the problem statement. 

There’s one more step that you can do in your journey toward abstraction. At the moment, the filterApples method works only for Apple. But you can also abstract on the List type to go beyond the problem domain you’re thinking of, as shown:

In [10]:
public interface Predicate<T> {
    boolean test(T t);
}

In [11]:
public <T> List<T> filter(List<T> list, Predicate<T> p) {
	List<T> result = new ArrayList<>();
	    for(T e: list) {
			if(p.test(e)) { 
				result.add(e);
			} 
		}
    return result;
}

You can now use the method filter with a List of bananas, oranges, Integers, or Strings! Here’s an example, using lambda expressions:

In [12]:
List<Apple> redApples = filter(inventory, (Apple apple) -> "red".equals(apple.getColor()));

redApples

[Apple{color='red', weight=220}]

In [13]:
var numbers = List.of(1,5,6,2,3,9);
List<Integer> evenNumbers = filter(numbers, (Integer i) -> i % 2 == 0);

evenNumbers

[6, 2]

_Isn’t it cool? You’ve managed to find the sweet spot between flexibility and concise-
ness, which wasn’t possible prior to Java 8!_

## Real-world examples

You’ve now seen that behavior parameterization is a useful pattern to easily adapt to changing requirements. This pattern lets you encapsulate a behavior (a piece of code) and parameterize the behavior of methods by passing and using these behaviors you create (for example, different predicates for an Apple)

__Sorting with a Comparator__

java.util.Comparator
```
public interface Comparator<T> {
    int compare(T o1, T o2);
}
```

In [14]:
var inventory = getInventory();

inventory

[Apple{color='green', weight=80}, Apple{color='green', weight=255}, Apple{color='red', weight=220}]

In [15]:
import java.util.Comparator;
inventory.sort(new Comparator<Apple>() {
    public int compare(Apple a1, Apple a2) {
        return a1.getWeight().compareTo(a2.getWeight()); 
    }
});

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

In [16]:
var inventory = getInventory();

inventory

[Apple{color='green', weight=80}, Apple{color='green', weight=255}, Apple{color='red', weight=220}]

In [17]:
inventory.sort((Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight()));

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__Executing a block of code with Runnable__

java.lang.Runnable
```
public interface Runnable {
     void run();
}

```

In [18]:
import java.lang.Runnable;
Thread t = new Thread(new Runnable() {
    public void run() {
        System.out.println("Hell o world"); }
});

t.start();

Hell o world


In [19]:
Thread t = new Thread(() -> System.out.println("Hell o world"));
t.start();

Hell o world


__Returning a result using Callable__

java.util.concurrent.Callable 
```
public interface Callable<V> {
    V call();
}
```

In [20]:
import java.util.concurrent.Callable;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
import java.util.concurrent.Future;

ExecutorService executorService = Executors.newCachedThreadPool();
Future<String> threadName = executorService.submit(new Callable<String>() {
	@Override
	public String call() throws Exception {
		return Thread.currentThread().getName();
	}
});

threadName.get();

pool-1-thread-1

In [21]:
Future<String> threadName = executorService.submit(() -> Thread.currentThread().getName());
threadName.get();

pool-1-thread-1

## Summary

* Behavior parameterization is the ability for a method to take multiple different behaviors as parameters and use them internally to accomplish different behaviors.

* Behavior parameterization lets you make your code more adaptive to changing requirements and saves on engineering efforts in the future.

* Passing code is a way to give new behaviors as arguments to a method. But it’s verbose prior to Java 8. Anonymous classes helped a bit before Java 8 to get rid of the verbosity associated with declaring multiple concrete classes for an inter- face that are needed only once.

* The Java API contains many methods that can be parameterized with different behaviors, which include sorting, threads, and GUI handling.